In [18]:
import pandas as pd
import os
import numpy as np

In [2]:
df = pd.read_csv(os.path.abspath('../input/standardized-diff-cleaned-featured-2008-2017.csv'))

In [3]:
df.columns.values

array(['p_number_of_matches_used', 'direct', 'o_number_of_matches_used',
       'player_id', 'opponent_id', 'tournament_name', 'date', 'p_matches',
       'fs_diff', 'w1sp_diff', 'w2sp_diff', 'wrp_diff', 'tpw_diff',
       'tmw_diff', 'aces_per_game_diff', 'df_per_game_diff',
       'bp_won_diff', 'wsp_diff', 'completeness_diff', 'serve_adv_diff'],
      dtype=object)

Drop unused features for prediction :

In [5]:
df = df.drop(['p_number_of_matches_used', 'o_number_of_matches_used', 'player_id', 'opponent_id', 'tournament_name', 'date'], axis=1)

In [8]:
X_train = df.drop(['p_matches'], axis=1).copy()
Y_train = df['p_matches']

In [31]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

estimator = LogisticRegression()
selector = RFE(estimator, n_features_to_select=1, step=1)
selector = selector.fit(X_train, Y_train)
selector.support_

array([False, False, False, False, False, False, False, False, False,
       False, False, False,  True])

In [32]:
selector.ranking_

array([ 9, 12,  7, 13,  4,  8,  5,  6, 11, 10,  3,  2,  1])

In [37]:
pd.DataFrame({'ranking': X_train.columns.values},index=selector.ranking_).sort_index()

,ranking
1,serve_adv_diff
2,completeness_diff
3,wsp_diff
4,wrp_diff
5,tmw_diff
6,aces_per_game_diff
7,w1sp_diff
8,tpw_diff
9,direct
10,bp_won_diff


import test dataset to evaluate with logistic loss :

In [39]:
test_df = pd.read_csv(os.path.abspath('../input/standardized-diff-cleaned-featured-2018.csv'))

In [40]:
test_df = test_df.drop(['p_number_of_matches_used', 'o_number_of_matches_used', 'player_id', 'opponent_id', 'tournament_name', 'date'], axis=1)

In [41]:
X_test = df.drop(['p_matches'], axis=1).copy()
Y_test = df['p_matches']

In [44]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

Y_pred = logreg.predict(X_test)

In [56]:
from sklearn.metrics import classification_report, log_loss, confusion_matrix

In [55]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

         0.0       0.68      0.68      0.68     26101
         1.0       0.68      0.68      0.68     26101

    accuracy                           0.68     52202
   macro avg       0.68      0.68      0.68     52202
weighted avg       0.68      0.68      0.68     52202



In [48]:
log_loss(Y_test, Y_pred)

11.179148710154807

In [57]:
confusion_matrix(Y_test, Y_pred)

array([[17653,  8448],
       [ 8448, 17653]], dtype=int64)